## Must do

1. exclude those who have a date lost to follow-up below May 2017 based on data-field 191 (done in extract features)

In [72]:
from IPython.display import display, HTML
import pandas as pd
from lxml import etree
import time
import re
import matplotlib.pyplot as plt
import random
import codecs
#import dask.dataframe as dd
#import dask.array as da
import numpy as np
import pickle
from datetime import datetime
from UKBB import UKBDicts,preprocess,tidy_coded_fieldID,save_obj




from bs4 import BeautifulSoup
import requests
import unicodedata

ImportError: cannot import name 'save_obj' from 'UKBB' (/home/mo/Env_vars_DL/UKBB.py)

In [2]:
#PATH_data=r"/home/workspace/datasets/ukbiobank/ukbiobank_yajiez/dataset/ukb26438.csv"
#PATH_dict=r"/home/workspace/datasets/ukbiobank/ukbiobank_yajiez/dataset/ukb26438.html"

PATH_data=r"/home/workspace/datasets/ukbiobank/biobank_milad_34943/ukb34943.csv"
PATH_dict=r"/home/mo/ukb34943.html"

# 2. Read data as pd for correctness check

## 2.1 read datafields by categories in numpy

In [3]:
ukb=UKBDicts(PATH_data,PATH_dict,"file")

categories,UID_to_dscrpt,fieldID_to_ctgry_dict=ukb.categories,ukb._UID_to_dscrpt,ukb._fieldID_to_ctgry_dict

Loaded a dictionary with 8226 field ID:category entries
Parsed 1827 unique field IDs from the UKBiobank html file
Time elapsed for extracting unique field IDs: 3.28003010712564


/home/mo/Env_vars_DL/UKBB.py:180: UserWarning: Some of the fieldIDs were not not found in self._UID_to_descrpt dictionary. Therefore their categories are not included in self.categories. See self.fieldID_no_category             for a list of these
  warnings.warn("Some of the fieldIDs were not not found in self._UID_to_descrpt dictionary. Therefore their categories are not included in self.categories. See self.fieldID_no_category             for a list of these")


In [ ]:
tic = time.perf_counter()
column_names = pd.read_csv(PATH_data, dtype='unicode',nrows=1).columns
df = pd.read_csv(PATH_data,dtype='unicode',engine='python')
#df=next(df)
toc = time.perf_counter()
print('Time to read the data: '+str(toc-tic))
df

# 3 Extract outcomes

## 3.1 41270 & 41280 (primary and secondary)

In [11]:
outcomes=['I20-I25 Ischaemic heart diseases',
'I30-I52 Other forms of heart disease',
'I70-I79 Diseases of arteries, arterioles and capillaries'
'I20.0 Unstable angina',
'I20.9 Angina pectoris, unspecified',
'I21 Acute myocardial infarction',
'I25.1 Atherosclerotic heart disease',
'I48 Atrial fibrillation and flutter', 
'I50 Heart failure']

# section below needs to be updated when the issue with pandas bool indexing has been resolved
## I think this can be solved by converting the whole dataframe to datatime. at the moment some are datatime and some np.nan, converting all to datetime will convert Nan to NaT and then the conditional indexing can happen

In [48]:
selected_categories={'Summary Diagnoses':['include','41270']}
process_config={'41270':{'type':'string'}}

df_temp=preprocess(selected_categories,ukb.categories,ukb._UID_to_dscrpt,df,process_config)


selected_categories={'Summary Diagnoses':['include','41280']}
process_config={'41280':{'type':'string'}}
df_temp_date=preprocess(selected_categories,ukb.categories,ukb._UID_to_dscrpt,df,process_config)

selected_categories1={fieldID_to_ctgry_dict['53'][1]:['include','53']}
process_config1={'53':{'type':'string'}}

df_date_assessment=pd.to_datetime(preprocess(selected_categories1,ukb.categories,
                                             ukb._UID_to_dscrpt,df,process_config1)['53-0.0'],format='%Y-%m-%d').to_frame()

#Outcomes
I20_25= ['I'+str(i) for i in range(20,26)]
I30_52=['I'+str(i) for i in range(30,53)]
#I70_79=['I'+str(i) for i in range(70,80)]
#outcomes_list=I20_25+I30_52+I70_79
outcomes_list=I20_25+I30_52+["G45","I63","I64"]

#find the bool_indx when one of the outcomes is found in df_temp
cols_dict={df_temp.columns.values[i]:df_temp_date.columns.values[i] for i in range(df_temp.shape[1])}
bool_indxs=df_temp.apply(lambda col: col.str.contains('|'.join(outcomes_list), na=False), axis=1)
#the corresponding bool indxing df for the dates dataframe
bool_indxs_date=bool_indxs.rename(columns=cols_dict)


#df_temp_date=df_temp_date[bool_indxs_date].apply(pd.to_datetime)
"""This line above used to work but due to an error in pandas it doesn't, revert pack when pandas error has been fixed"""
df_temp_date_X=pd.DataFrame(columns=df_temp_date.columns, index=df_temp_date.index)
for col in bool_indxs_date.columns:
    if bool_indxs_date.loc[:,col].sum()!=0:
        df_temp_date_X.loc[bool_indxs_date.loc[:,col],col]=df_temp_date.loc[bool_indxs_date.loc[:,col],col].apply(pd.to_datetime)
    

the_earliest_outcome_diagnosis=df_temp_date_X.apply(pd.to_datetime).min(axis=1)# the first occurance of the disease(any of the ones specified in outcomes)

From the category Summary Diagnoses the following where selected: 
 	 {'41270'}
	 FID 41270:  Diagnoses - ICD10Uses data-coding 259
From the category Summary Diagnoses the following where selected: 
 	 {'41280'}
	 FID 41280:  Date of first in-patient diagnosis - ICD10
From the category Reception the following where selected: 
 	 {'53'}
	 FID 53:  Date of attending assessment centre


In [53]:
newdf=pd.DataFrame(index=df_temp.index,columns=['outcome_months','Exclude_outcomes','outcomes'])
newdf.loc[the_earliest_outcome_diagnosis<df_date_assessment['53-0.0'],'Exclude_outcomes']=1

newdf.loc[the_earliest_outcome_diagnosis>=df_date_assessment['53-0.0'],'outcomes']=1

newdf['outcome_months']=the_earliest_outcome_diagnosis.sub(df_date_assessment['53-0.0'])
newdf['outcome_months']=newdf['outcome_months']/np.timedelta64(1, 'M')

#replace MI_months nans with the duration of time between may 2017 and date of attending the assessment centre
indx=newdf.index[newdf['outcome_months'].isna()]
newdf.loc[indx,'outcome_months']=(datetime(2017,5,1)-df_date_assessment['53-0.0'])/np.timedelta64(1, 'M')
newdf.fillna(0,inplace=True)

In [54]:
selected_outcome_categories_tidy={'CVDs':['outcomes','outcome_months']}
selected_outcome_categories_tidy_columns={'outcomes':{"col_name":['outcome'],"type":"binary"},
                                         'outcome_months':{"col_name":['outcome_months'],"type":"int"}}
selected_outcome_categories_exclusions=["Exclude_outcomes"]

## 3.2 40000 40001 & 40002 (death registry) 

In [56]:
word='death'
print(ukb.search_description(word))
print(fieldID_to_ctgry_dict['40002'])
categories['Death register']

Field ID: 1807 _ Father's age at deathUses data-coding 100435

Field ID: 3526 _ Mother's age at deathUses data-coding 100360

Field ID: 4501 _ Non-accidental death in close genetic familyUses data-coding 100259

Field ID: 20437 _ Thoughts of death during worst depressionUses data-coding 505

Field ID: 20530 _ Witnessed sudden violent deathUses data-coding 533

Field ID: 40000 _ Date of death

Field ID: 40001 _ Underlying (primary) cause of death: ICD10Uses data-coding 19

Field ID: 40002 _ Contributory (secondary) causes of death: ICD10Uses data-coding 19

Field ID: 40007 _ Age at death

Field ID: 40010 _ Description of cause of death

Field ID: 40018 _ Death record formatUses data-coding 261

Field ID: 40020 _ Death record originUses data-coding 1970

None
['Contributory (secondary) causes of death: ICD10', 'Death register']


['40000: Date of death',
 '40001: Underlying (primary) cause of death: ICD10Uses data-coding 19',
 '40002: Contributory (secondary) causes of death: ICD10Uses data-coding 19',
 '40007: Age at death',
 '40010: Description of cause of death',
 '40018: Death record formatUses data-coding 261',
 '40020: Death record originUses data-coding 1970']

In [57]:
selected_categories={'Death register':['include','40001','40002']}
process_config={'40001':{'type':'string'},'40002':{'type':'string'}}

df_temp=preprocess(selected_categories,ukb.categories,ukb._UID_to_dscrpt,df,process_config)

selected_categories={'Death register':['include','40000']}
process_config={'40000':{'type':'string'}}
df_date_death=preprocess(selected_categories,ukb.categories,ukb._UID_to_dscrpt,df,process_config)['40000-0.0'].apply(pd.to_datetime)

From the category Death register the following where selected: 
 	 {'40002', '40001'}
	 FID 40002:  Contributory (secondary) causes of death: ICD10Uses data-coding 19
	 FID 40001:  Underlying (primary) cause of death: ICD10Uses data-coding 19
From the category Death register the following where selected: 
 	 {'40000'}
	 FID 40000:  Date of death


In [58]:
bool_indxs=df_temp.apply(lambda col: col.str.contains('|'.join(outcomes_list), na=False), axis=1)
df_date_death_outcome=df_date_death[bool_indxs.any(axis=1)] #this is death due to outcomes of interest (CVDs defined above)
#find indices (patients) in newdf where death has been to do the outcomes (df_date_death_outcome.index) but have not had previous diagnoses, 
#i.e. outcomes=0 and exclude_outcomes=0
indxs_to_be_updated=(newdf.iloc[df_date_death_outcome.index]['Exclude_outcomes']==0) & \
(newdf.iloc[df_date_death_outcome.index]['outcomes']==0)

indxs_to_be_updated=indxs_to_be_updated[indxs_to_be_updated] #This contains only the true values (those that need to be updated)


#now update the outcomes, and outcome_months of these indices
newdf.loc[indxs_to_be_updated.index]['outcomes']=1
newdf.loc[indxs_to_be_updated.index]['outcome_months']=df_date_death_outcome.loc[indxs_to_be_updated.index].sub(df_date_assessment.loc[indxs_to_be_updated.index,'53-0.0'])/np.timedelta64(1, 'M')

newdf['death']=df_date_death


In [59]:
word='follow'
print(ukb.search_description(word))
print(fieldID_to_ctgry_dict['40002'])
categories['Death register']

Field ID: 190 _ Reason lost to follow-upUses data-coding 1965

Field ID: 191 _ Date lost to follow-up

Field ID: 20086 _ Type of special diet followedUses data-coding 76

Field ID: 20554 _ Actions taken following self-harmUses data-coding 1423

None
['Contributory (secondary) causes of death: ICD10', 'Death register']


['40000: Date of death',
 '40001: Underlying (primary) cause of death: ICD10Uses data-coding 19',
 '40002: Contributory (secondary) causes of death: ICD10Uses data-coding 19',
 '40007: Age at death',
 '40010: Description of cause of death',
 '40018: Death record formatUses data-coding 261',
 '40020: Death record originUses data-coding 1970']

# Other Exclusions

## Statin use

In [60]:
newdf['Exclude_statin_use']=0
statin_codes=['1140861958','1141146234','1140888594','1140888648','1141192410'] #simvastatin atorvastatin fluvastatin pravastatin rosuvastatin
temp_bool_array=np.isin(df[[col for col in df.columns if '20003-0' in col]],statin_codes).any(axis=1)
indxs_temp=df.loc[temp_bool_array,[col for col in df.columns if '20003-0' in col]].index
newdf.loc[indxs_temp,'Exclude_statin_use']=1


selected_outcome_categories_exclusions.append('Exclude_statin_use')

TypeError: '(Int64Index([     4,     12,     18,     20,     31,     33,     34,     49,
                50,     52,
            ...
            502482, 502483, 502488, 502490, 502499, 502504, 502510, 502516,
            502519, 502526],
           dtype='int64', length=78033), 'Exclude_statin_use')' is an invalid key

In [74]:
import pickle
newdf.to_csv(r'/home/mo/Env_vars_DL/tidy_data_tiles/Outcomes.csv')

save_obj(selected_outcome_categories_tidy,'selected_outcome_categories_tidy')
save_obj(selected_outcome_categories_tidy_columns,'selected_outcome_categories_tidy_columns')
save_obj(selected_outcome_categories_exclusions,'selected_outcome_categories_exclusions')